In [2]:
import pandas as pd
import os

# Load the cleaned dataset from Step 5
clean_path = "/home/danial/Data Science/Churn Prediction/Data/Cleaned/telco_clean.csv"
df = pd.read_csv(clean_path)

# Quick check of shape and first rows
print("Shape:", df.shape)
df.head()


Shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Create average monthly charges (TotalCharges divided by tenure)
# For tenure=0, use MonthlyCharges directly
df['AvgChargesPerMonth'] = df.apply(
    lambda x: x['MonthlyCharges'] if x['tenure']==0 else x['TotalCharges']/x['tenure'],
    axis=1
)

df[['tenure','MonthlyCharges','TotalCharges','AvgChargesPerMonth']].head()


,tenure,MonthlyCharges,TotalCharges,AvgChargesPerMonth
0,1,29.85,29.85,29.850000
1,34,56.95,1889.50,55.573529
2,2,53.85,108.15,54.075000
3,45,42.30,1840.75,40.905556
4,2,70.70,151.65,75.825000


In [4]:
# Difference between actual TotalCharges and expected (tenure * MonthlyCharges)
df['ChargeDiff'] = df['TotalCharges'] - (df['tenure'] * df['MonthlyCharges'])

# Relative difference percentage (avoid division by zero by using where)
df['ChargeDiffPerc'] = df['ChargeDiff'] / df['TotalCharges'].where(df['TotalCharges']!=0, 1)

df[['tenure','MonthlyCharges','TotalCharges','ChargeDiff','ChargeDiffPerc']].head()


,tenure,MonthlyCharges,TotalCharges,ChargeDiff,ChargeDiffPerc
0,1,29.85,29.85,0.00,0.000000
1,34,56.95,1889.50,-46.80,-0.024768
2,2,53.85,108.15,0.45,0.004161
3,45,42.30,1840.75,-62.75,-0.034089
4,2,70.70,151.65,10.25,0.067590


In [5]:
# Difference between actual TotalCharges and expected (tenure * MonthlyCharges)
df['ChargeDiff'] = df['TotalCharges'] - (df['tenure'] * df['MonthlyCharges'])

# Relative difference percentage (avoid division by zero by using where)
df['ChargeDiffPerc'] = df['ChargeDiff'] / df['TotalCharges'].where(df['TotalCharges']!=0, 1)

df[['tenure','MonthlyCharges','TotalCharges','ChargeDiff','ChargeDiffPerc']].head()


,tenure,MonthlyCharges,TotalCharges,ChargeDiff,ChargeDiffPerc
0,1,29.85,29.85,0.00,0.000000
1,34,56.95,1889.50,-46.80,-0.024768
2,2,53.85,108.15,0.45,0.004161
3,45,42.30,1840.75,-62.75,-0.034089
4,2,70.70,151.65,10.25,0.067590


In [6]:
# Group tenure into bins to capture customer lifecycle stages
bins = [-1, 12, 24, 48, df['tenure'].max()]
labels = ['0-12','12-24','24-48','48+']
df['tenure_group'] = pd.cut(df['tenure'], bins=bins, labels=labels)

df[['tenure','tenure_group']].head(10)


,tenure,tenure_group
0,1,0-12
1,34,24-48
2,2,0-12
3,45,24-48
4,2,0-12
5,8,0-12
6,22,12-24
7,10,0-12
8,28,24-48
9,62,48+


In [7]:
# Replace "No internet service"/"No phone service" with "No" for simplicity
cols_internet = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
for c in cols_internet:
    if c in df.columns:
        df[c] = df[c].replace('No internet service','No')

if 'MultipleLines' in df.columns:
    df['MultipleLines'] = df['MultipleLines'].replace('No phone service','No')

# Count active services per customer (Yes=1)
service_cols = [c for c in ['PhoneService','MultipleLines'] + cols_internet if c in df.columns]
df['NumServices'] = df[service_cols].apply(lambda r: (r=='Yes').sum(), axis=1)

# Flag for having internet at all
df['HasInternet'] = (df['InternetService']!='No').astype(int)

df[['NumServices','HasInternet']].head()


,NumServices,HasInternet
0,1,1
1,3,1
2,3,1
3,3,1
4,1,1


In [11]:
# Create numeric churn flag (Yes=1, No=0)
df['ChurnFlag'] = df['Churn'].map({'Yes':1,'No':0})

# Encode simple binary columns
binary_cols = ['Partner','Dependents','PhoneService','PaperlessBilling',
               'MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection',
               'TechSupport','StreamingTV','StreamingMovies']
binary_cols = [c for c in binary_cols if c in df.columns]
df[binary_cols] = df[binary_cols].replace({'Yes':1,'No':0})

# Encode gender to numeric (Male=1, Female=0)
df['gender_Male'] = df['gender'].map({'Male':1,'Female':0})

df[['Churn','ChurnFlag','gender','gender_Male']].head()


,Churn,ChurnFlag,gender,gender_Male
0,No,0,Female,0
1,No,0,Male,1
2,Yes,1,Male,1
3,No,0,Male,1
4,Yes,1,Female,0


In [12]:
# Save the feature-engineered dataset
save_path = "/home/danial/Data Science/Churn Prediction/Data/Processed/telco_featured.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
df.to_csv(save_path, index=False)
print("Saved to:", save_path)

# Quick check of new features
new_cols = ['AvgChargesPerMonth','ChargeDiff','ChargeDiffPerc','tenure_group','NumServices','HasInternet','ChurnFlag','gender_Male']
print("New features:", [c for c in new_cols if c in df.columns])


Saved to: /home/danial/Data Science/Churn Prediction/Data/Processed/telco_featured.csv
New features: ['AvgChargesPerMonth', 'ChargeDiff', 'ChargeDiffPerc', 'tenure_group', 'NumServices', 'HasInternet', 'ChurnFlag', 'gender_Male']
